In [49]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [50]:
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [51]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [52]:
train.shape, test.shape

((1460, 81), (1459, 80))

დავდროპოთ გამოუსადეგარი ან არასრული ინფორმაციის შემცველი სვეტები

In [53]:
cols_to_drop = ['Alley', 'PoolQC', 'Fence', 'MiscFeature', 'Id']
train.drop(columns=cols_to_drop, inplace=True)
test.drop(columns=cols_to_drop, inplace=True)

In [54]:
train.shape, test.shape

((1460, 76), (1459, 75))

ამოვიღოთ categorical და numerical ცვლადები, დავჰენდლოთ NaN მნიშვნელობები

In [55]:
num_cols = train.select_dtypes(include='number').drop(columns='SalePrice').columns
cat_cols = train.select_dtypes(include='object').columns

train[num_cols] = train[num_cols].fillna(train[num_cols].mean())
train[cat_cols] = train[cat_cols].fillna('None')

test_num_cols = test.select_dtypes(include='number').columns
test_cat_cols = test.select_dtypes(include='object').columns

test[test_num_cols] = test[test_num_cols].fillna(test[test_num_cols].mean())
test[test_cat_cols] = test[test_cat_cols].fillna('None')

One-Hot Encoding

In [56]:
train_encoded = pd.get_dummies(train.drop('SalePrice', axis=1))
test_encoded = pd.get_dummies(test)

train_encoded, test_encoded = train_encoded.align(test_encoded, join='left', axis=1, fill_value=0)

Train-ის და Test-ის დასპლიტვა და წრფივი რეგრესიით ტრეინინგი

In [57]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

y = train['SalePrice']
X = train_encoded

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

RMSE-ით შეფასება

In [58]:
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("Validation RMSE:", rmse)

Validation RMSE: 29604.04659942617


In [59]:
!pip install dagshub mlflow

In [60]:
import dagshub
import mlflow

In [61]:
dagshub.init(repo_owner='LukaVakhania', repo_name='Hw1-House-Prices', mlflow=True)

Initialized MLflow to track repo "LukaVakhania/Hw1-House-Prices"

Repository LukaVakhania/Hw1-House-Prices initialized!

In [62]:
import mlflow.sklearn

with mlflow.start_run():
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_metric("rmse", rmse)
    mlflow.sklearn.log_model(model, "model")

2025/04/13 20:08:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run chill-grub-518 at: https://dagshub.com/LukaVakhania/Hw1-House-Prices.mlflow/#/experiments/0/runs/cafb3d0b15874f83bdffc34b6505d27f
🧪 View experiment at: https://dagshub.com/LukaVakhania/Hw1-House-Prices.mlflow/#/experiments/0
